Generating Training Data, Validation Data and Test data from FitzhughNagumo Model using classical Euler method
Author:Manal Alghmadi


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

In [2]:
def fhn_equations(U, W, K):
  dU_dt = K * U * (U - 0.4) * (1 - U) - W
  dW_dt = 0.5 * (0.2 * U - 0.8 * W)
  return dU_dt, dW_dt

In [3]:
def euler_solver(u0, w0, K, t_max, dt):
    """Solve FHN model using Euler method"""
    t = np.arange(0, t_max, dt)
    u = np.zeros_like(t)
    w = np.zeros_like(t)
    u[0], w[0] = u0, w0
    for i in range(1, len(t)):
        du_dt, dw_dt = fhn_equations(u[i - 1], w[i - 1], K)
        u[i] = u[i - 1] + du_dt * dt
        w[i] = w[i - 1] + dw_dt * dt
    return t, u, w

In [4]:
def generate_fhn_data(num_samples,seed=None):
    """Generate data for different initial conditions and K values"""
    data = []
    t_max = 100
    dt = 0.01
    if seed is not None:
        np.random.seed(seed)  # Set the random seed for reproducibility
    for i in range((num_samples)):
      u0 = np.random.uniform(.4,0.6)  # Random number from 0 to 0.6
      w0 = np.random.uniform(0,.2)   # Random number from 0 to 0.6
      K = np.random.uniform(2,2.7) 
      t, u, w = euler_solver(u0, w0, K, t_max, dt)
      data.append({'t': t,'u0': u0, 'w0': w0, 'K': K, 'u': u, 'w': w})
    return data

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming `data` is your complete dataset
training_data, validation_data = train_test_split(generate_fhn_data(num_samples=1000,seed=42), test_size=0.2, random_state=42)
test_data, validation_data = train_test_split(validation_data, test_size=0.5, random_state=42)

In [ ]:
# Randomly select four samples to plot
num_samples = len(training_data)
random_indices = np.random.choice(num_samples, size=4, replace=False)

# Create a figure with 4 subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 10))  # 2 rows, 2 columns

# Flatten the array of axes for easy iteration
axs = axs.flatten()

# Loop through the randomly selected samples and plot their u and w values
for i, idx in enumerate(random_indices):
    axs[i].plot(training_data[idx]['t'], training_data[idx]['u'], label='u', color='blue')
    axs[i].plot(training_data[idx]['t'], training_data[idx]['w'], label='w', color='orange', linestyle='dotted')
    
    # Set labels and title for each subplot
    axs[i].set_xlabel('Time')
    axs[i].set_ylabel('Value')
    axs[i].set_title(f"FHN Model Sample: u0={training_data[idx]['u0']:.2f}, w0={training_data[idx]['w0']:.2f}, K={training_data[idx]['K']:.2f}")
    axs[i].legend()
    axs[i].grid()

plt.tight_layout()
plt.show()

In [ ]:
#helper method to transform generated data to tensors and return it as batches
def Transform_to_Tensors (Data,batch_size):
    t = torch.FloatTensor(Data[0]['t']).reshape(-1, 1).requires_grad_(True)
    U0 = torch.FloatTensor([Data[0]['u0']]).reshape(-1, 1) #Single value
    W0 = torch.FloatTensor([Data[0]['w0']]).reshape(-1, 1)
    K = torch.FloatTensor([Data[0]['K']]).reshape(-1, 1)
    U_true = torch.FloatTensor(Data[0]['u']).reshape(-1, 1)
    W_true = torch.FloatTensor(Data[0]['w']).reshape(-1, 1)
    # Create a TensorDataset
    dataset = TensorDataset(t, U0.expand_as(t), W0.expand_as(t), K.expand_as(t), U_true, W_true)

    # Create DataLoader for batching
    batch_size = 1000  # You can adjust this value
    data_loader= DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return data_loader

In [ ]:
test_data = Transform_to_Tensors(test_data,1000)
training_data = Transform_to_Tensors(training_data,1000)
validation_data = Transform_to_Tensors(validation_data,1000)